In [1]:
%matplotlib inline  

import matplotlib
matplotlib.use("Qt4Agg")
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import sqlite3
import tables
import py7D
import requests
import resource
import sevendigital_preview
from audioanalytics import FluctuationPattern
from main import create_fluctuation_data
resource.setrlimit(resource.RLIMIT_NOFILE, (65536, 65536))
os.chdir('msd_data/MillionSongSubset/data')

/usr/lib/python2.7/dist-packages/matplotlib/__init__.py:1314: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [ ]:
def create_db():
    conn = sqlite3.connect("processed_data.db")
    c = conn.cursor()
    c.execute('''CREATE TABLE tracks (msd_id text, track_7digitalid integer, jam_id text, som_cluster integer, igmn_cluster integer, audio_path text, h5_path text, genre text)''')
    conn.commit()
    conn.close()
create_db()

In [2]:
def jam_mapping(path):
    mapping = {}
    with open(path) as f:
        for line in f:
            values = line.split('\t')
            mapping[values[1].strip('\n')] = values[0]
    return mapping

In [3]:
jam_ids = jam_mapping('../../jam_to_msd.tsv')
#jam_ids

In [5]:
no_audio = []
no_jam = []
conn = sqlite3.connect("processed_data.db")
c = conn.cursor()

for(directory, _,files) in os.walk('.'):
    for h5 in glob.glob(os.path.join(directory,"*.h5")):
        print h5
        f = tables.open_file(h5, mode='r')
        track_id = f.root.analysis.songs.cols.track_id[0]
        audio_name = track_id +".mp3"
        audio_path = os.path.abspath(os.path.join(directory,audio_name))
        seven_id = f.root.metadata.songs.cols.track_7digitalid[0]
        #create db entry and get audiofile
        if not (os.path.exists(audio_path)):
            audio_url = sevendigital_preview.get_url(h5)#audio_url = py7D.preview_url(seven_id)
            r = requests.get(audio_url, headers={'Connection':'close'})
            if r.ok:
                #write audio file
                with open(audio_path, 'wb') as handle:
                    for block in r.iter_content(8*1024):
                        handle.write(block)
                #create db_entry
                query = "INSERT INTO tracks (msd_id, track_7digitalid, jam_id, audio_path, h5_path) VALUES ('%s',%d,'%s','%s','%s')"
                if jam_ids.has_key(track_id):
                    query = query % (track_id, seven_id, jam_ids[track_id], audio_path, os.path.abspath(h5))
                else:
                    query = query % (track_id, seven_id, -1, audio_path, h5)
                    no_jam.append(track_id)
                c.execute(query)
                conn.commit()
            else:
                no_audio.append(track_id)
            r.connection.close()
        else:
            print "skip file alread"
        f.close()
        
conn.close()
print "done"

./A/X/X/TRAXXZP128F427ACA6.h5
skip
./A/X/X/TRAXXTG128F932B324.h5
skip
./A/X/X/TRAXXJZ12903CF1731.h5
skip
./A/X/X/TRAXXQA128F9301855.h5
skip
./A/X/X/TRAXXOL128F92E7529.h5
skip
./A/X/X/TRAXXWU12903CF2094.h5
./A/X/X/TRAXXGI128F14688C7.h5
skip
./A/X/X/TRAXXVN128E0791AE4.h5
skip
./A/X/X/TRAXXEI128F4293139.h5
skip
./A/X/X/TRAXXUM128F4244507.h5
skip
./A/X/X/TRAXXTA128F146BCA7.h5
skip
./A/X/X/TRAXXYI12903CC4FDC.h5
skip
./A/X/X/TRAXXIT128F4263075.h5
skip
./A/X/X/TRAXXKJ128F92FDCD2.h5
./A/X/E/TRAXEHW128F423E590.h5
skip
./A/X/E/TRAXEQE128EF35CB1C.h5
./A/X/E/TRAXEBW128F4292391.h5
./A/X/E/TRAXEAE128F42930AA.h5
./A/X/E/TRAXECH128F421B47F.h5
skip
./A/X/E/TRAXEDF12903CD5DFB.h5
skip
./A/X/E/TRAXEID128F92E0922.h5
skip
./A/X/E/TRAXEML128F934BC04.h5
./A/X/E/TRAXEAY128F931C3D0.h5
./A/X/E/TRAXEWH12903CF66CB.h5
./A/X/E/TRAXEUB12903CBAA99.h5
./A/X/E/TRAXEYV128F932CD95.h5
skip
./A/X/T/TRAXTAU128F930A7EF.h5
skip
./A/X/T/TRAXTAM12903CF3556.h5
skip
./A/X/T/TRAXTDT128F4243F22.h5
./A/X/T/TRAXTTX128F9301092.h5
skip


In [ ]:
conn = sqlite3.connect("processed_data.db")
c = conn.cursor()
c.execute("SELECT * FROM tracks WHERE audio_path IS NOT NULL")
c.fetchone()   

In [ ]:
a = FluctuationPattern("/home/kayibal/sc-recom/code/msd_data/MillionSongSubset/data/A/X/X/TRAXXZP128F427ACA6.mp3", skip_frames = 0, chunk=16, Hz=11025)
fm = a.get_feature_matrix()
FluctuationPattern.plot_frame(fm)

In [ ]:
create_fluctuation_data('.')